## **1.2_1.6_DATASET_masters_merge_european_maize_files.ipynb**

### GOALS of this script:
* part one of whole merging workflow for the masters
* numbering in accordance to workflow_europe_maize_dataset_prep_masters
* combination/check of:
    * merging_script_final-Copy1.ipynb
    * merging_script_final.ipynb
    * the corresponding number steps
* OUT: final_columns_xxx_for_masters.csv

### *Import packages*

In [ ]:
import gzip
import allel
import pandas as pd
import numpy as np
import tskit
import tsinfer
import sys
import json
import csv
from IPython.display import SVG
from IPython.display import HTML
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook
from progressbar import ProgressBar
sns.set_style('white')
sns.set_style('ticks')

## **1.2 Read in all dataset files as hapmap format**

### *Read in Unterseer_2016 files*

#### *Read in hapmap file*

In [ ]:
unterseer_2016_landraces_hapmap=pd.read_csv("/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/unterseer_2016_landraces_hapmap.hmp.txt",sep="\t")
unterseer_2016_landraces_hapmap

In [ ]:
unterseer_2016_elite_hapmap=pd.read_csv("/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/unterseer_2016_elite_hapmap.hmp.txt",sep="\t")
unterseer_2016_elite_hapmap

### *Read in Mayer_2020 files*

#### *Read in .txt file* 

In [ ]:
#read in the .txt file from the /data/1_raw/4_Mayer_2017 folder 
mayer_2020_dhlines=pd.read_csv('/Users/kschul38/Documents/tsinfer-project/data/1_raw/3_Mayer_2020/Mayer_et_al_genotypes_DHlines_600k_raw.txt.gz', sep=" ")
#markers already used as row names 

In [ ]:
mayer_2020_dhlines

### *Read in Mayer_2017 files*

#### *Read in .txt file*

In [ ]:
#read in the .txt file from the /data/1_raw/4_Mayer_2017 folder 
mayer_2017_landraces=pd.read_csv('/Users/kschul38/Documents/tsinfer-project/data/1_raw/4_Mayer_2017/TUM-PLANTBREEDING_Maize600k_35landraces_952individuals_raw.txt', sep=" ")
mayer_2017_landraces=mayer_2017_landraces.reset_index()
#markers already used as row names 

In [ ]:
mayer_2017_landraces

# **1.3 Read in .vcf files**

### *Read in the Axiom Array marker, the IDs and the marker quality classifications*

#### *Read in Unterseer_2016_landraces vcf file*

In [ ]:
unterseer_2016_landraces_vcf= allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_landraces.vcf',fields=['variants/ID','variants/AD', 'variants/CMT'] , log=sys.stdout)

In [ ]:
#unterseer_2016_landraces_vcf

In [ ]:
#unterseer_landraces_dataframe = allel.vcf_to_dataframe('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_landraces.vcf', fields= ['variants/ID','variants/AD', 'variants/CMT'])
#unterseer_landraces_dataframe = unterseer_landraces_dataframe.reset_index(drop=True)

#### *Read in Unterseer_2016_elite vcf file*

In [ ]:
unterseer_2016_elite_vcf= allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_elitelines.vcf',fields=['variants/ID','variants/AD', 'variants/CMT'], log=sys.stdout)

In [ ]:
#unterseer_2016_elite_vcf

In [ ]:
#unterseer_elite_dataframe = allel.vcf_to_dataframe('/Users/kschul38/Documents/tsinfer-project/data/1_raw/5_Unterseer_2016/TUM-PLANTBREEDING_Maize600k_elitelines.vcf',fields=['variants/ID','variants/AD', 'variants/CMT'])
#unterseer_elite_dataframe = unterseer_elite_dataframe.reset_index(drop=True)

## **1.4 Compare samples between all datasets**

### *1.4.1 Compare sample names without prefix - overlap?*

**Only relevant combination - Unterseer_2016_landraces & Mayer_2017_landraces**

*Unterseer_2016_landraces*

In [ ]:
#get the samples in the hapmap
unterseer_2016_landraces_hapmap_columns=list(unterseer_2016_landraces_hapmap.columns)
unterseer_2016_landraces_hapmap_columns=unterseer_2016_landraces_hapmap_columns[11:]
#unterseer_2016_landraces_hapmap_columns
print(len(unterseer_2016_landraces_hapmap_columns))

In [ ]:
unterseer_landraces_pop=Counter([elem.split(".", 1)[0] for elem in unterseer_2016_landraces_hapmap_columns])

In [ ]:
unterseer_landraces_pop_array=np.array(list(unterseer_landraces_pop.items()))

In [ ]:
unterseer_landraces_pop_dataframe=pd.DataFrame(data=unterseer_landraces_pop_array, columns=["Populations", "Numbers_unterseer"])
#unterseer_landraces_pop_dataframe

*Mayer_2017_landraces*

In [ ]:
##get the samples in the hapmap
mayer_2017_landraces_columns=list(mayer_2017_landraces.columns)
mayer_2017_landraces_columns=mayer_2017_landraces_columns[6:]
#mayer_2017_landraces_columns
print(len(mayer_2017_landraces_columns))

In [ ]:
mayer_landraces_pop=Counter([elem.split(".", 1)[0] for elem in mayer_2017_landraces_columns])

In [ ]:
mayer_landraces_pop_array=np.array(list(mayer_landraces_pop.items()))

In [ ]:
mayer_landraces_pop_dataframe=pd.DataFrame(data=mayer_landraces_pop_array, columns=["Populations", "Numbers_mayer"])

In [ ]:
result = pd.merge(unterseer_landraces_pop_dataframe,mayer_landraces_pop_dataframe, on="Populations", how="left")

In [ ]:
result=result.fillna(0)

In [ ]:
result["Numbers_unterseer"]=result["Numbers_unterseer"].astype(int)

In [ ]:
result["Numbers_mayer"]=result["Numbers_mayer"].astype(int)

In [ ]:
result["Difference"] =result["Numbers_unterseer"] - result["Numbers_mayer"]

In [ ]:
#result

## **1.5 Merge the quality & marker name columns back into the Unterseer tables**

### *1.5.1 Add suffix*

#### *Add suffix to hapmap files*

In [ ]:
unterseer_2016_landraces_hapmap_suffix=unterseer_2016_landraces_hapmap.add_suffix('_2016_landraces')
unterseer_2016_landraces_hapmap_suffix

In [ ]:
unterseer_2016_elite_hapmap_suffix=unterseer_2016_elite_hapmap.add_suffix('_2016_elite')
unterseer_2016_elite_hapmap_suffix

In [ ]:
mayer_2020_dhlines_suffix=mayer_2020_dhlines.add_suffix('_2020_dh')
#mayer_2020_dhlines_suffix=mayer_2020_dhlines_suffix.reset_index()
mayer_2020_dhlines_suffix

In [ ]:
mayer_2017_landraces_suffix=mayer_2017_landraces.add_suffix('_2017_landraces')
#mayer_2017_landraces_suffix=mayer_2017_landraces_suffix.reset_index()
mayer_2017_landraces_suffix

In [ ]:
type(mayer_2017_landraces_suffix)

#### *Rename Unterseer .vcf columns to also contain the suffix*

In [ ]:
unterseer_2016_landraces_vcf.keys()

In [ ]:
unterseer_2016_elite_vcf.keys()

In [ ]:
unterseer_2016_landraces_vcf = pd.DataFrame(data=unterseer_2016_landraces_vcf)
#unterseer_2016_landraces_vcf = unterseer_2016_landraces_vcf.set_index("variants/ID")
unterseer_2016_landraces_vcf = unterseer_2016_landraces_vcf.rename(columns={"variants/AD": "variants/AD_2016_landraces","variants/CMT": "variants/CMT_2016_landraces", "variants/ID": "rs#_2016_landraces" })

In [ ]:
unterseer_2016_landraces_vcf

In [ ]:
unterseer_2016_elite_vcf = pd.DataFrame(data=unterseer_2016_elite_vcf)
#unterseer_2016_elite_vcf = unterseer_2016_elite_vcf.set_index("variants/ID")
unterseer_2016_elite_vcf = unterseer_2016_elite_vcf.rename(columns={"variants/AD": "variants/AD_2016_elite","variants/CMT": "variants/CMT_2016_elite", "variants/ID": "rs#_2016_elite"})

In [ ]:
type(unterseer_2016_elite_vcf)

### *1.5.2 Merge based on marker ID*

#### *Unterseer_2016_elite*

In [ ]:
unterseer_2016_elite_complete=pd.merge(unterseer_2016_elite_hapmap_suffix, unterseer_2016_elite_vcf, on="rs#_2016_elite")
unterseer_2016_elite_complete

#### *Unterseer_2016_landraces*

In [ ]:
unterseer_2016_landraces_complete=pd.merge(unterseer_2016_landraces_hapmap_suffix, unterseer_2016_landraces_vcf, on="rs#_2016_landraces")
unterseer_2016_landraces_complete

### *1.5.3 reorder the columns in the table*

#### *Unterseer_2016_landraces*

In [ ]:
reorder_landraces=unterseer_2016_landraces_hapmap_suffix.columns
reorder_landraces_geno=reorder_landraces[11:]
reorder_landraces_geno=reorder_landraces_geno.to_list()
#reorder_landraces_geno

In [ ]:
unterseer_2016_landraces_info=["rs#_2016_landraces","variants/AD_2016_landraces","variants/CMT_2016_landraces","alleles_2016_landraces","chrom_2016_landraces","pos_2016_landraces","strand_2016_landraces"]

In [ ]:
unterseer_2016_landraces_reordered_list = unterseer_2016_landraces_info + reorder_landraces_geno
#unterseer_2016_landraces_reordered_list

In [ ]:
unterseer_2016_landraces_reordered=unterseer_2016_landraces_complete[unterseer_2016_landraces_reordered_list]

In [ ]:
unterseer_2016_landraces_reordered

#### *Unterseer_2016_elite*

In [ ]:
reorder_elite=unterseer_2016_elite_hapmap_suffix.columns
reorder_elite_geno=reorder_elite[11:]
reorder_elite_geno=reorder_elite_geno.to_list()
#reorder_elite_geno

In [ ]:
unterseer_2016_elite_info=["rs#_2016_elite","variants/AD_2016_elite","variants/CMT_2016_elite","alleles_2016_elite","chrom_2016_elite","pos_2016_elite","strand_2016_elite"]

In [ ]:
unterseer_2016_elite_reordered_list = unterseer_2016_elite_info + reorder_elite_geno
#unterseer_2016_elite_reordered_list

In [ ]:
unterseer_2016_elite_reordered=unterseer_2016_elite_complete[unterseer_2016_elite_reordered_list]

In [ ]:
unterseer_2016_elite_reordered

## **1.6 merge the AGPv4 poitions into the Unterseer_2016 files**

### *1.6.1A Get the columns from the Mayer_2017 papers - for landraces*

*Get the index column, the chromosme pos and pos for both the AGPv2 and the AGPv4 from the Mayer_2017 paper*

In [ ]:
mayer_2017_landraces_pos_columns_for_landraces=mayer_2017_landraces_suffix.columns
mayer_2017_landraces_pos_columns_for_landraces=mayer_2017_landraces_pos_columns_for_landraces[0:5]
mayer_2017_landraces_pos_columns_for_landraces

*Rename the index, the chromosome pos and pos for AGPv2 to be the same as in Unterseer_landraces*

In [ ]:
mayer_2017_landraces_pos_for_landraces=mayer_2017_landraces_suffix[mayer_2017_landraces_pos_columns_for_landraces]

In [ ]:
mayer_2017_landraces_pos_for_landraces

In [ ]:
mayer_2017_landraces_pos_for_landraces=mayer_2017_landraces_pos_for_landraces.rename(columns={"index_2017_landraces":"variants/AD_2016_landraces","chr_v2_2017_landraces":"chrom_2016_landraces","pos_v2_2017_landraces":"pos_2016_landraces"})
mayer_2017_landraces_pos_for_landraces

### *1.6.2A Merge based on marker ID and AGPv2 chromosome and position - for landraces*

In [ ]:
unterseer_2016_landraces_new_pos=pd.merge(mayer_2017_landraces_pos_for_landraces,unterseer_2016_landraces_reordered,on=["variants/AD_2016_landraces", "chrom_2016_landraces","pos_2016_landraces"])
unterseer_2016_landraces_new_pos

### *1.6.3A Drop the columns that are not needed - for landraces*

*Drop the AGPv2 columns*

In [ ]:
unterseer_2016_landraces_right_pos=unterseer_2016_landraces_new_pos.drop(columns=['chrom_2016_landraces', 'pos_2016_landraces','rs#_2016_landraces'])
unterseer_2016_landraces_right_pos

### *1.6.1B Get the columns from the Mayer_2017 papers - for elite*

*Get the index column, the chromosme pos and pos for both the AGPv2 and the AGPv4 from the Mayer_2017 paper*

In [ ]:
mayer_2017_landraces_pos_columns_for_elite=mayer_2017_landraces_suffix.columns
mayer_2017_landraces_pos_columns_for_elite=mayer_2017_landraces_pos_columns_for_elite[0:5]
mayer_2017_landraces_pos_columns_for_elite

*Rename the index, the chromosome pos and pos for AGPv2 to be the same as in Unterseer_landraces*

In [ ]:
mayer_2017_landraces_pos_for_elite=mayer_2017_landraces_suffix[mayer_2017_landraces_pos_columns_for_elite]

In [ ]:
mayer_2017_landraces_pos_for_elite

In [ ]:
mayer_2017_landraces_pos_for_elite=mayer_2017_landraces_pos_for_elite.rename(columns={"index_2017_landraces":"variants/AD_2016_elite","chr_v2_2017_landraces":"chrom_2016_elite","pos_v2_2017_landraces":"pos_2016_elite"})
mayer_2017_landraces_pos_for_elite

### *1.6.2B Merge based on marker ID and AGPv2 chromosome and position - for elite*

In [ ]:
unterseer_2016_elite_new_pos=pd.merge(mayer_2017_landraces_pos_for_elite,unterseer_2016_elite_reordered,on=["variants/AD_2016_elite", "chrom_2016_elite","pos_2016_elite"])
unterseer_2016_elite_new_pos

### *1.6.3B Drop the columns that are not needed - for elite*

*Drop the AGPv2 columns*

In [ ]:
unterseer_2016_elite_right_pos=unterseer_2016_elite_new_pos.drop(columns=['chrom_2016_elite', 'pos_2016_elite','rs#_2016_elite'])
unterseer_2016_elite_right_pos

## **OUTPUT**

In [ ]:
final_columns_unterseer_2016_elite=unterseer_2016_elite_right_pos

In [ ]:
final_columns_unterseer_2016_landraces=unterseer_2016_landraces_right_pos

In [ ]:
final_columns_mayer_2017_landraces=mayer_2017_landraces_suffix.drop(columns=['chr_v2_2017_landraces', 'pos_v2_2017_landraces'])
final_columns_mayer_2017_landraces

In [ ]:
final_columns_mayer_2020_dhlines=mayer_2020_dhlines_suffix
final_columns_mayer_2020_dhlines

In [ ]:
xxxx

### **Write to file**

In [ ]:
final_columns_unterseer_2016_elite.to_csv('final_columns_unterseer_2016_elite_for_masters.csv', sep="\t", index = False)

In [ ]:
final_columns_unterseer_2016_landraces.to_csv('final_columns_unterseer_2016_landraces_for_masters.csv', sep="\t", index = False)

In [ ]:
final_columns_mayer_2017_landraces.to_csv('final_columns_mayer_2017_landraces_for_masters.csv', sep="\t", index = False)

In [ ]:
final_columns_mayer_2020_dhlines.to_csv('final_columns_mayer_2020_dhlines_for_masters.csv', sep="\t", index = False)